# A) Prediction problem: Linear regression

## A.1) Exploratory data analysis

### Insight 1
I investigated the distribution and shape of the response variable, `price`, to see whether there are any potential transformations that could be suitable. We can see that the distribution is very right-skewed, so a transformation such as the log transformation would be helpful in using for a predictive regression model. 

### Insight 2

Not only is the response variable, `price`, more suited to be transformed, but other predictors being transformed such as `number_of_reviews` can also be helpful for building the regression model. When we visualize the distribution by creating a histogram, we can see that the distribution is right-skewed, so using a log transformation would also help build a more accurate model.

By the same vein, other similar variables such as `number_of_reviews_ltm`, `number_of_reviews_l30d`, and `calculated_host_listings_count_private_rooms` also have a similar right-skewed distribution, so they can also be log transformed.

### Insight 3

There are some useful categorical variables that seem like they help with predicting `price`. However, a good amount of those have levels within the categorical variable that do not align with the test data. To build a model containing those variables, we need to ensure that the levels of those variables match between the train and test data. This suggests that combining levels within these categories would be helpful, as hyperspecific levels of a category may not yield much insight in the data sets.

### Insight 4

Another way I went about doing exploratory data analysis is analyzing the overall correlations of the data set, particularly with respsect to `log_price`.  

From the correlations, `accommodates`, `beds`, and `bath` variables seem to be the highest correlated predictors with `price`. As such, this provided a good basis for these predictors to be included in the regression model. 

### Insight 5

A useful method I used to filter out more predictors to include in the final model was analyzing the correlations with `price` and `log_price`. Variables like `host_acceptance_rate` and `host_response_rate` were not nearly as highly correlated as I thought with price. Therefore, I removed predictors that had really low correlations with the response `log_price`.

Furthermore, I also checked the correlations between predictors to see if multicollinearity occured. For instance, `minimum_minimum_nights` was highly correlated with `minimum_nights`, and `availability_90` was highly correlated with `availability_365` so I removed those predictors that were along the same vein. The others like `maximum_minimum_nights` and `maximum_nights_avg_ntm` also happened to have very low correlation, so it helped further narrow down useful predictors to include in the final model.

### Insight 6

To further improve predictive accuracy, I also examined any potential influential points. I checked for any extreme outliers and removed any observations that had a `price` higher than 10000. In addition, I found any of the most influential points and removed a high leverage point that had very high number of `reviews`.

## A.2) Data cleaning / preparation

In [45]:
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime as dt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer

In [500]:
train = pd.read_csv('./regression-problem/train_regression.csv')
test = pd.read_csv('./regression-problem/test_regression.csv')

In [501]:
# Define a function to categorize the property types
def categorize_property(property_type):
    if 'Entire' in property_type:
        return 'Entire Home/Apartment'
    elif 'Private' in property_type:
        return 'Private Room'
    elif 'Shared' in property_type:
        return 'Shared Accommodation'
    elif property_type in ['Room in hotel', 'Room in boutique hotel', 'Boat']:
        return 'Specialty Accommodations'
    else:
        return 'Other'

In [502]:
# overall function to clean training and test data
def clean_data(df):
    
    # Remove $ from response variable and convert to float in training data
    if 'price' in df.columns:
        df.price = df.price.replace('[\$,]', '', regex=True).astype(float)
        
    # replace missing values of numeric variables wtih the median
    numeric_columns = df.select_dtypes(include=['number']).columns
    df[numeric_columns] = df[numeric_columns].apply(lambda x: x.fillna(x.median()))

    # replace missing values of categorical variables with the mode 
    categorical_columns = df.select_dtypes(include=['object']).columns
    df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])
    
    # log transform response variable for training data and drop price
    if 'price' in df.columns:
        df['log_price'] = np.log(df['price'])
    
    # replace any 0 values to 1 so that it can go through log transformation
    df['beds'] = df['beds'].replace(0, .01)
    df['accommodates'] = df['accommodates'].replace(0, .01)
    df['number_of_reviews'] = df['number_of_reviews'].replace(0, .01)
    df['reviews_per_month'] = df['reviews_per_month'].replace(0, .01)
    df['number_of_reviews_ltm'] = df['number_of_reviews_ltm'].replace(0, .01)
    df['number_of_reviews_l30d'] = df['number_of_reviews_l30d'].replace(0, .01)
    df['host_total_listings_count'] = df['host_total_listings_count'].replace(0, .01)
    df['host_listings_count'] = df['host_listings_count'].replace(0, .01)
    df['calculated_host_listings_count_private_rooms'] = df['calculated_host_listings_count_private_rooms'].replace(0, .01)
    df['calculated_host_listings_count_shared_rooms'] = df['calculated_host_listings_count_shared_rooms'].replace(0, .01)
    df['calculated_host_listings_count_entire_homes'] = df['calculated_host_listings_count_entire_homes'].replace(0, .01)
    
    df['log_beds'] = np.log(df.beds)
    df['log_accommodates'] = np.log(df.accommodates)
    df['log_reviews'] = np.log(df.number_of_reviews)
    df['log_reviews_per_month'] = np.log(df.reviews_per_month)
    df['log_reviews_ltm'] = np.log(df.number_of_reviews_ltm)
    df['log_reviews_l30d'] = np.log(df.number_of_reviews_l30d)
    df['log_host_total_listings_count'] = np.log(df.host_total_listings_count)
    df['log_host_listings_count'] = np.log(df.host_listings_count)
    df['log_host_listings_count_private_rooms'] = np.log(df.calculated_host_listings_count_private_rooms)
    df['log_host_listings_count_shared_rooms'] = np.log(df.calculated_host_listings_count_shared_rooms)
    df['log_host_listings_count_entire_homes'] = np.log(df.calculated_host_listings_count_entire_homes)

    # calculate the number of days since the host became a host
    df['host_since'] = pd.to_datetime(df['host_since'])
    current_date = dt.now()
    df['host_since_days'] = (current_date - df['host_since']).dt.days
    
    # calculate days since first/last review
    df['first_review'] = pd.to_datetime(df['first_review'], errors='coerce')
    df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

    df['first_review_days'] = (current_date - df['first_review']).dt.days
    df['last_review_days'] = (current_date - df['last_review']).dt.days
    
    # make response_rate and acceptance_rate into numeric dtype
    df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype('float')
    df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').astype('float')
    
    # subgroup property_type (similar levels as room_type so discard room predictor)
    df['property_cats'] = df['property_type'].apply(categorize_property)
    
    # extract numeric values from the 'bathrooms' column
    df['bath_numeric'] = df['bathrooms_text'].str.extract('(\d+\.*\d*)', expand=False).astype(float)

    # handle "Half-bath" by assigning a numeric value of 0.5
    df['bath_numeric'] = df.apply(lambda row: 0.5 if 'half' in row['bathrooms_text'].lower() \
                                  else row['bath_numeric'], axis=1)
    
    # to binary
    df.host_is_superhost = df.host_is_superhost.replace({'t': 1, 'f': 0})
    df.host_identity_verified = df.host_identity_verified.replace({'t': 1, 'f': 0})
    df.host_has_profile_pic = df.host_has_profile_pic.replace({'t': 1, 'f': 0})
    df.has_availability = df.has_availability.replace({'t': 1, 'f': 0})
    df.instant_bookable = df.instant_bookable.replace({'t': 1, 'f': 0})

    # drop the modified/redundant columns
    df.drop(columns = ['host_since', 'first_review', 'last_review', 'property_type', 'bathrooms_text', \
                       'number_of_reviews', 'reviews_per_month', 'number_of_reviews_ltm', \
                       'number_of_reviews_l30d', 'host_total_listings_count', 'host_listings_count', \
                      'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', \
                       'calculated_host_listings_count_entire_homes', 'host_id'], inplace = True)
    
    # drop predictors that have low corr with log_price and high corr with others to help remove multi-collinearity
    df.drop(columns = ['first_review_days', 'last_review_days', 'host_acceptance_rate', 'host_response_rate', 
                       'availability_60', 'availability_90', 'minimum_minimum_nights', \
                       'maximum_maximum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', \
                       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'], inplace = True)

In [503]:
clean_data(train)
clean_data(test)

In [504]:
# extract mean price of each host_location predictor
host_loc_avg_prices = train.groupby('host_location')['price'].mean().reset_index()
host_loc_avg_prices['avg_host_location_price'] = host_loc_avg_prices['price']
train = pd.merge(train, host_loc_avg_prices[['host_location', 'avg_host_location_price']], on='host_location', how='left')

# extract mean price of each host_neighbourhood predictor
host_neigh_avg_prices = train.groupby('host_neighbourhood')['price'].mean().reset_index()
host_neigh_avg_prices['avg_host_neighbourhood_price'] = host_neigh_avg_prices['price']
train = pd.merge(train, host_neigh_avg_prices[['host_neighbourhood', 'avg_host_neighbourhood_price']], on='host_neighbourhood', how='left')

# extract mean price of each neighbourhood_cleansed predictor
neigh_avg_prices = train.groupby('neighbourhood_cleansed')['price'].mean().reset_index()
neigh_avg_prices['avg_neighbourhood_price'] = neigh_avg_prices['price']
train = pd.merge(train, neigh_avg_prices[['neighbourhood_cleansed', 'avg_neighbourhood_price']], on='neighbourhood_cleansed', how='left')

In [505]:
# extract mean price of each host_location predictor for test data
test = pd.merge(test, host_loc_avg_prices[['host_location', 'avg_host_location_price']], on='host_location', how='left')

# extract mean price of each host_neighbourhood predictor for test data
test = pd.merge(test, host_neigh_avg_prices[['host_neighbourhood', 'avg_host_neighbourhood_price']], on='host_neighbourhood', how='left')

# extract mean price of each neighbourhood_cleansed predictor for test data
test = pd.merge(test, neigh_avg_prices[['neighbourhood_cleansed', 'avg_neighbourhood_price']], on='neighbourhood_cleansed', how='left')

In [506]:
# drop the categorical predictors that we used right above
train = train.drop(columns = ['host_neighbourhood', 'neighbourhood_cleansed', 'host_location'])
test = test.drop(columns = ['host_neighbourhood', 'neighbourhood_cleansed', 'host_location'])

In [507]:
# filter out extreme outliers
train = train[train.price < 10000]

In [508]:
# drop the most influential point
train = train.drop(index = 2850)

In [509]:
# OHE the remaining categorical variables
host_response_time_dummies = pd.get_dummies(train['host_response_time'], prefix='host_response_time')
train = pd.concat([train, host_response_time_dummies], axis = 1)

host_response_time_dummies = pd.get_dummies(test['host_response_time'], prefix='host_response_time')
test = pd.concat([test, host_response_time_dummies], axis = 1)

In [510]:
host_verifications_dummies = pd.get_dummies(train['host_verifications'], prefix='host_verifications')
train = pd.concat([train, host_verifications_dummies], axis = 1)

host_verifications_dummies = pd.get_dummies(test['host_verifications'], prefix='host_verifications')
test = pd.concat([test, host_verifications_dummies], axis = 1)

In [511]:
room_type_dummies = pd.get_dummies(train['room_type'], prefix='room_type')
train = pd.concat([train, room_type_dummies], axis = 1)

room_type_dummies = pd.get_dummies(test['room_type'], prefix='room_type')
test = pd.concat([test, room_type_dummies], axis = 1)

In [512]:
property_cats_dummies = pd.get_dummies(train['property_cats'], prefix='property_cats')
train = pd.concat([train, property_cats_dummies], axis = 1)

property_cats_dummies = pd.get_dummies(test['property_cats'], prefix='property_cats')
test = pd.concat([test, property_cats_dummies], axis = 1)

In [513]:
train = train.drop(columns = ['host_response_time', 'host_verifications', 'room_type', 'property_cats'])
test = test.drop(columns = ['host_response_time', 'host_verifications', 'room_type', 'property_cats'])

In [514]:
# variable spacing
train.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
test.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

In [515]:
# replace missing values of numeric variables with mean in test from added predictors
numeric_columns = test.select_dtypes(include=['number']).columns
test[numeric_columns] = test[numeric_columns].apply(lambda x: x.fillna(x.mean()))

In [516]:
# set response and predictors for scaling
y_train = train.log_price
X_train = train.drop(columns = ['log_price', 'price', 'id'])
X_test = test.drop(columns = ['id'])

In [517]:
# poly features
poly = PolynomialFeatures(degree = 2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)

In [518]:
# scale the variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

## A.3) Developing the model

### Step 1: Transformations
As touched upon in the first section, there were some variables that were identified to be more suitable for a log transformation due to having a right-skewed distribution. The variables that ended up getting log transformed were `price`, `number_of_reviews`, and the other review variables.

Furthermore, I tested out higher-order polynomial transformation terms to add more flexibility to the model. This was useful for the variables that are pretty highly correlated with `log_price`, and I ended up using the second-order polynomial features when setting up the training and test data sets.

Just replacing the transformed log versions of the `review` variables instead of the original non-transformed predictors improved the model, which led me to keep them in the model. The correlations of the log versions also backed this up.

### Step 2: Variable Selection
Given that there is a wide array of predictor variables in the train data, it made sense to first think intuitively about predictor variables that make sense in a model predicting `price`. Firstly, `accommodates` stood out as higher number of accommodates should correspond in an increase in price. These conclusions are also supported by the findings in EDA when correlations were examined. The log versions of `accommodates` and `beds` were also highly correlated with `log_price`, so they were also included in the Ridge regression. I chose to use Ridge regression as there is a decent number of predictors resulting from getting all the two-way interactions betweent the predictors.

Initially, intuition and trial and error was the main strategy I used when I went about building the model. For this final model, I decided to use Ridge regression as there were many predictors that had potential relationships with `log_price`. I also used `PolynomialFeatures` to get all the two-way interactions between the variables. Therefore, using Ridge to shrink these coefficients was the method of model building and variable selection. 

### Step 3: Significant Interactions

As with the variable selection step, I mainly used intuition and trial and error at first to test significant interactions and whether or not they improved the model. I tested `beds` and `bath_numeric`, as it makes sense for there to be an interaction between the two to impact the price (when there are more beds than baths, when they are even, etc.). Interactions between different types of `review` scores also intuitively made sense, as a listing could have better reviews in some areas over others. Furthermore, `property_cats` (`property_type` in broader categories) and `bath_numeric` (`bathrooms_text` numerized) seemed to be significant. 

Using `PolynomialFeatures` was a more efficient and thorough way of running all the different two-way interactions between the chosen variables. I ran it with a degree of 2 and put the training set through Ridge regression to scale all the coefficients accordingly.

## A.4) Model equation

### Ridge

In [266]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LogisticRegressionCV, LogisticRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict

In [519]:
alphas = np.logspace(-1,3,50)
model = RidgeCV(alphas=alphas, cv=10)

model.fit(X_train_scaled, y_train)

RidgeCV(alphas=array([1.00000000e-01, 1.20679264e-01, 1.45634848e-01, 1.75751062e-01,
       2.12095089e-01, 2.55954792e-01, 3.08884360e-01, 3.72759372e-01,
       4.49843267e-01, 5.42867544e-01, 6.55128557e-01, 7.90604321e-01,
       9.54095476e-01, 1.15139540e+00, 1.38949549e+00, 1.67683294e+00,
       2.02358965e+00, 2.44205309e+00, 2.94705170e+00, 3.55648031e+00,
       4.29193426e+00, 5.17947468e+0...
       9.10298178e+00, 1.09854114e+01, 1.32571137e+01, 1.59985872e+01,
       1.93069773e+01, 2.32995181e+01, 2.81176870e+01, 3.39322177e+01,
       4.09491506e+01, 4.94171336e+01, 5.96362332e+01, 7.19685673e+01,
       8.68511374e+01, 1.04811313e+02, 1.26485522e+02, 1.52641797e+02,
       1.84206997e+02, 2.22299648e+02, 2.68269580e+02, 3.23745754e+02,
       3.90693994e+02, 4.71486636e+02, 5.68986603e+02, 6.86648845e+02,
       8.28642773e+02, 1.00000000e+03]),
        cv=10)

In [520]:
model.alpha_

268.26957952797244

In [521]:
pred = np.exp(model.predict(X_test_scaled))

In [522]:
id = test.id.values
predicted = pred
submission = pd.DataFrame({'id': id, 'predicted': predicted})
submission = submission.reset_index(drop=True)
submission.to_csv('regression_submission.csv', index=False)